# Bayesian Ridge Regression Part 2

### Multiple Features

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# yahoo finance is used to fetch data 
import yfinance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2018-08-27'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,3.85,3.98,3.84,3.95,3.95,20548400
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700


In [3]:
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,0)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200,1,1,1,0.012658
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300,1,1,1,0.032500
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100,0,1,0,0.012106
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700,0,0,0,0.000000
2014-01-09,4.20,4.23,4.05,4.09,4.09,30667600,0,0,1,-0.021531


In [4]:
dataset.shape

(1170, 10)

In [5]:
X = np.asanyarray(dataset[['Open','High','Low', 'Volume', 'Increase_Decrease', 'Buy_Sell_on_Open', 'Buy_Sell', 'Returns']])
y = np.asanyarray(dataset[['Adj Close']])

In [6]:
from sklearn.linear_model import BayesianRidge, LinearRegression

# Fit the Bayesian Ridge Regression and an OLS for comparison
model = BayesianRidge(compute_score=True)
model.fit(X, y)

BayesianRidge(compute_score=True)

In [7]:
model.coef_

array([-3.71811100e-01,  7.54346179e-01,  6.18115091e-01, -5.16918356e-10,
        8.91043617e-03,  2.73262150e-02, -8.23675662e-03,  9.98167208e-01])

In [8]:
model.scores_

array([-2952.66924316,  1057.53631316,  1121.57060587,  1121.58018745,
        1121.58019083,  1121.58019307])

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [11]:
model = BayesianRidge(compute_score=True)
model.fit(X_train, y_train)

BayesianRidge(compute_score=True)

In [12]:
model.coef_

array([-4.17226302e-01,  7.63815386e-01,  6.54267580e-01, -5.91194634e-10,
        5.17901154e-03,  2.47593159e-02, -7.39514755e-03,  9.03019952e-01])

In [13]:
model.scores_

array([-2363.16808463,   828.80515885,   894.65473886,   894.66445236,
         894.66445545,   894.66445753])

In [14]:
y_pred = model.predict(X_test)

In [15]:
from sklearn.metrics import mean_squared_error
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

The rmse of prediction is: 0.09287337824617113


In [16]:
print('Bayesian Ridge Regression Score:', model.score(X_test, y_test))

Bayesian Ridge Regression Score: 0.9996452147933678
